In [1]:
!pip install altair -qqq
!pip install pyarrow -qqq


# **Задача**

Сегодня вы - владельцы большого таксопарка и зарабатываете на посуточной аренде. 

- В вашем гараже есть машины разных моделей и классов, которые вы сдаете в аренду. 
- Доход от сдачи в аренду каждой модели автомобиля разный, он фиксирован внутри сезона. 

В последние годы бизнес растёт. 

- Прибавляются новые машины, но и расходы на обслуживание увеличиваются. 
- У вас и ваших партнёров есть предположение, что можно расти еще быстрее и зарабатывать еще больше денег

От вас требуется:

---

1. Преобразовать, провалидировать и проаналилизировать данные
2. Понять, что происходит в бизнесе: рост или падение и за счёт чего
3. Сформулировать стратегию и цели на следующий год: на чём стоит сделать акценты и к чему идти?
4. Полученные результаты представить в виде презентации решения на слайдах

---

# **Фичи**

время

- `dt` - месяц
- `dim` - количество дней в месяце

Тип машин

- `car_model` - модель машины
- `car_type` - тип авто (эконом, комфорт, бизнес)

количество машин

- `car_qnty` - число таких машин в парке
- `new_cars` - число приобретенных таксопарком машин 
- `bad_cars` - число машин, которые в этом месяце **весь месяц стояли в ремонте**
- `active_cars` - число активных машин за месяц (add)

стоимость

- `price per day` - стоимость аренды за 1 день 

In [2]:
# ok!
# df['repair price'] * df['bad_cars'] * df['dim']
# df['money_outcome_repairs']

### **active_cars**

Доступные машины для аренды (active_cars) [car_qnty - bad_cars]

- `days available` - машино-дней, которые **были доступны для сдачи** в аренду (если было 10 машин и 30 дней, то 300 машино-дней)
- **`days on line`** - сколько машино-дней автомобили **сдавались в аренду** 
- **`days off line`** - сколько машино-дней автомобили были доступны, но простояли 

### **not active cars**

недоступные машины для аренды (bad_cars)

- `days_in_repair` - число машино-дней, в которые машины находились в ремонте


### **expenses**

- `repair price` - стоимость 1 дня ремонта 1 машины (у каждой модели свой сервис)
- `money_outcome_repairs` - расходы на ремонт 
- `money_outcome_other` - прочие расходы (страховка, недобросовестные водители и тд)
- `new_cars_outcome` - расходы на приобретение новых машин такого типа

In [3]:
import numpy as np
import pandas as pd
np.float = float  # Patch deprecated alias
import openpyxl
import plotly.express as px

# load data 
path = "Задание #2 (для студентов).xlsx"
workbook = openpyxl.load_workbook(path)

df = pd.DataFrame(workbook['датасет'].values)
df.columns = df.iloc[0,:]
df.drop([0],axis=0,inplace=True)
df = df.reset_index(drop=True)
df.columns

Index(['dt', 'car_model', 'car_type', 'car_qnty', 'new_cars', 'bad_cars',
       'days available', 'days on line', 'days_in_repair', 'price per day',
       'repair price', 'money_outcome_repairs', 'money_outcome_other',
       'new_cars_outcome'],
      dtype='object', name=0)

# **Валидация данных**

## 1. Поправки тип колонок

- Так же удаляем строки без информации
- Преобразуем колонки в целы числа где можем, остальные это категориальные, либо с "пропуском"

In [4]:
# basic validation & column type adjustments
df = df.dropna(how='all',axis=0) # validation (remove missing rows)

df['new_cars'] = df['new_cars'].astype('Int64')
df['bad_cars'] = df['bad_cars'].astype('Int64')
df['car_qnty'] = df['car_qnty'].astype('Int64')
df['repair price'] = df['repair price'].astype('Int64')
df['money_outcome_repairs'] = df['money_outcome_repairs'].astype('Int64')
df['money_outcome_other'] = df['money_outcome_other'].astype('Int64')
df['days_in_repair'] = df['days_in_repair'].astype('Int64')

## 2. Проверки содержания колонок

Данные содержат пропуски
- **no data** тип пропуска есть в данных!
- Пройдемся по всем уникальным значении и убедится что в колонке нет ошибок

In [5]:
df['dt'].unique() # days since 1899-12-30

array([44562.0, 44593.0, 44621.0, 44652.0, 44682.0, 44713.0, 44743.0,
       44774.0, 44805.0, 44835.0, 44866.0, 44896.0, 44927.0, 44958.0,
       44986.0, 45017.0, 45047.0, 45078.0, 45108.0, 45139.0, 45170.0,
       45200.0, 45231.0, 45261.0, 45292.0], dtype=object)

In [6]:
df['car_model'].unique()

array(['Logan', 'Solaris', 'Kia K5', 'JAC J7', 'E200'], dtype=object)

In [7]:
# print(df[['car_type']].to_markdown(tablefmt='simple'))
df['car_type'].unique()

array(['Econom', 'Comfort', 'Business'], dtype=object)

In [8]:
df['car_qnty'].unique()

<IntegerArray>
[31, 32, 33, 34, 38, 41, 42, 43, 44, 45, 46, 47, 48, 21, 22, 23, 24, 29, 30,
 11, 12, 13, 14,  7,  8, 16, 19, 26,  2,  3,  5,  6, 10]
Length: 33, dtype: Int64

In [9]:
df['new_cars'].unique() # Есть стандарный пропуск

<IntegerArray>
[1, 0, 4, 3, 5, <NA>, 2]
Length: 7, dtype: Int64

In [10]:
df['bad_cars'].unique()

<IntegerArray>
[3, 5, 1, 9, 2, 0, 4, 6, 7, 8]
Length: 10, dtype: Int64

In [11]:
df['days available'].unique() # no data present

array([868.0, 756.0, 992.0, 750.0, 1023.0, 930.0, 961.0, 100000.0, 1054.0,
       1036.0, 1209.0, 1200.0, 1240.0, 1230.0, 1333.0, 1290.0, 1364.0,
       1320.0, 1457.0, 620.0, 616.0, 682.0, 690.0, 744.0, 780.0, 837.0,
       810.0, 775.0, 806.0, 720.0, 660.0, 651.0, 279.0, 308.0, 372.0,
       330.0, 403.0, 341.0, 390.0, 364.0, 360.0, 434.0, 420.0, 248.0,
       155.0, 224.0, 240.0, 310.0, 450.0, 480.0, 496.0, 527.0, 532.0,
       589.0, 899.0, 62.0, 56.0, 31.0, 90.0, 93.0, 150.0, 186.0, 180.0,
       280.0, 'no data'], dtype=object)

In [12]:
df['days on line'].unique() # no data present

array([711.0, 740.0, 972.0, 622.0, 849.0, 846.0, 807.0, 942.0, 781.0,
       961.0, 818.0, 1022.0, 828.0, 1025.0, 1184.0, 960.0, 1091.0, 1119.0,
       1253.0, 1146.0, 1044.0, 1148.0, 1319.0, 1296.0, 564.0, 603.0,
       668.0, 662.0, 714.0, 780.0, 786.0, 777.0, 837.0, 778.0, 736.0,
       682.0, 773.0, 684.0, 728.0, 699.0, 627.0, 697.0, 594.0, 605.0,
       237.0, 187.0, 234.0, 267.0, 330.0, 303.0, 265.0, 'no data', 296.0,
       334.0, 366.0, 241.0, 322.0, 218.0, 313.0, 354.0, 302.0, 312.0,
       386.0, 316.0, 297.0, 344.0, 148.0, 331.0, 128.0, 212.0, 150.0,
       329.0, 351.0, 374.0, 491.0, 411.0, 500.0, 547.0, 501.0, 581.0,
       655.0, 620.0, 790.0, 744.0, 863.0, 1627.0, 48.0, 34.0, 24.0, 54.0,
       55.0, 90.0, 116.0, 139.0, 144.0, 124.0, 135.0, 130.0, 193.0, 221.0,
       225.0, 245.0, 314.0, 280.0, 290.0, 277.0, 286.0], dtype=object)

In [13]:
df['days_in_repair'].unique()

<IntegerArray>
[ 93, 140,  31, 270,  90,  62,   0,  28,  30,  60, 124,  56, 120, 180, 155,
 186, 210, 240, 279]
Length: 19, dtype: Int64

In [14]:
df['price per day'].unique() # no data present

array([2000.0, 100.0, 1900.0, 2900.0, 3200.0, 2800.0, 2600.0, 2200.0,
       'no data', 2100.0, 9000.0, 10000.0, 10001.0], dtype=object)

In [15]:
df['repair price'].unique()

<IntegerArray>
[ 1500,  1279,  1275,  1295,  1255,  1212,  1222,  1289,  1280,  1245,  1285,
  1268,  1192,  1198,  1077,  1012,   905,   960,   837,   881,   813,   885,
   838,   810,   828,  1740,  2500,  1717,  1760,  1797,  1796,  1775,  1752,
  1810,  1713,  1709,  1753,  1735,  1815,  1719,  1811,  1823,  1716,  1848,
  1708,  1754,  1783,  1731,  1798,  1720,  1721, 20000, 32000, 31000, 31001]
Length: 55, dtype: Int64

In [16]:
df['money_outcome_repairs'].unique()

<IntegerArray>
[139500, 210000,  46500, 405000, 135000,  93000,      0,  42000,  45000,
  90000, 186000,  39649,  40145,  37650, 109980,  79918,  39680,  77340,
  38595,  77100,  78616,  73904,  67088, 133548, 121440,  84165, 172800,
 129735, 163866, 170730, 137175, 201120, 225990, 231012, 107880,  70000,
  52200, 104400, 465000, 106454, 167121,  55025, 106206,  51270, 108686,
 156150, 168795, 107570,  96264, 112282,  54690,  53196, 158844, 108748,
  53490, 107322,  53940, 159960, 160053, 620000, 992000, 930000, 961000]
Length: 63, dtype: Int64

In [17]:
df['money_outcome_other'].unique()

<IntegerArray>
[128733, 103580, 141270, 101438, 132209, 114809, 127932, 110510, 124039,
 106688,
 ...
 139228, 116858, 141884, 120376, 147359, 132831, 141444, 148489, 128343,
 109353]
Length: 121, dtype: Int64

In [18]:
df['new_cars_outcome'].unique() # no data present

array([400000.0, 0.0, 1600000.0, 1200000.0, 380000.0, 1900000.0,
       2100000.0, 2600000.0, 5700000.0, 3800000.0, 'no data', 3000000.0,
       6000000.0, 9000000.0], dtype=object)

In [19]:
# replace 'no data' with pandas NA
for col in list(df.columns):
    try:
        df[col] = df[col].replace({'no data':pd.NA})
    except:
        pass
    
    try:
        df[col] = df[col].astype('Int64')
    except:
        pass

## 3. Поправка даты 

- колонка `dt` совпадает в первым днем месяцв, ie. поправим тип в yyyy-mm
- из даты подтянем и количество дней в этом месяце, `dim`

In [20]:
# 4) define interpretable format for dt
df['dt'] = df['dt'].astype('float')
df['dt'] = pd.to_datetime(df['dt'], unit='D', origin='1899-12-30')
df['dim'] = df['dt'].dt.days_in_month
df['dt'] = df['dt'].dt.strftime('%Y-%m')

In [21]:
df.head()

,dt,car_model,car_type,car_qnty,new_cars,bad_cars,days available,days on line,days_in_repair,price per day,repair price,money_outcome_repairs,money_outcome_other,new_cars_outcome,dim
0,2022-01,Logan,Econom,31,1,3,868,711,93,2000,1500,139500,128733,400000,31
1,2022-02,Logan,Econom,32,1,5,756,740,140,2000,1500,210000,103580,400000,28
2,2022-03,Logan,Econom,33,1,1,992,972,31,2000,1500,46500,141270,400000,31
3,2022-04,Logan,Econom,34,0,9,750,622,270,2000,1500,405000,101438,0,30
4,2022-05,Logan,Econom,34,0,1,1023,849,31,2000,1500,46500,132209,0,31


## 4. Обработка пропусков

Нам нужно будем делать ручные правки, и автоматические, на основе правил

Из лонгрида, раздел **Отсутствие данных (пропуски)**

> удалять пропуски при условии, что у вас достаточно данных и количество пропусков не превышает 5-10%. Если удалить больше, результаты могут исказиться, а точность анализа — снизиться;


In [22]:
df.isna().sum().to_frame().T

,dt,car_model,car_type,car_qnty,new_cars,bad_cars,days available,days on line,days_in_repair,price per day,repair price,money_outcome_repairs,money_outcome_other,new_cars_outcome,dim
0,0,0,0,0,3,0,1,1,0,1,0,0,0,1,0


In [23]:
# manual preprocessing
df.loc[44,'new_cars'] = 0
df.loc[91,'new_cars'] = 0
df.loc[92,'price per day'] = 2200
df.loc[92,'new_cars_outcome'] = 5700000
df.loc[116,'new_cars'] = 0

# automatic preprocessing based on column rule
df['active_cars'] = df['car_qnty'] - df['bad_cars'] # количество активных машин в этом месяце
df['days_available'] = df['active_cars'] * df['dim'] # требуется чтобы заполнить пропуск в колонке [days available]
df['days available'] = df.apply(lambda x: x['days_available'] if pd.isna(x['days available']) == True else x['days available'],axis=1)

In [24]:
df.isna().sum().to_frame().T

,dt,car_model,car_type,car_qnty,new_cars,bad_cars,days available,days on line,days_in_repair,price per day,repair price,money_outcome_repairs,money_outcome_other,new_cars_outcome,dim,active_cars,days_available
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


- We found a critical missing data row in our problem, as we are interested in examining the revenue values; days on line is critical. 
- No specific patterns can be found in the following data below

In [25]:
tt = df[df['car_model'] == 'Kia K5']
tt.loc[50:60]

,dt,car_model,car_type,car_qnty,new_cars,bad_cars,days available,days on line,days_in_repair,price per day,repair price,money_outcome_repairs,money_outcome_other,new_cars_outcome,dim,active_cars,days_available
50,2022-01,Kia K5,Comfort,11,1,2,279,237,62,2900,1740,107880,134368,2100000,31,9,279
51,2022-02,Kia K5,Comfort,12,0,1,308,187,28,2900,2500,70000,122000,0,28,11,308
52,2022-03,Kia K5,Comfort,12,0,0,372,234,0,2900,2500,0,100864,0,31,12,372
53,2022-04,Kia K5,Comfort,12,1,1,330,267,30,2900,1740,52200,103260,2100000,30,11,330
54,2022-05,Kia K5,Comfort,13,0,0,403,330,0,2900,1740,0,128501,0,31,13,403
55,2022-06,Kia K5,Comfort,13,0,2,330,303,60,2900,1740,104400,133234,0,30,11,330
56,2022-07,Kia K5,Comfort,13,0,2,341,265,62,2900,1740,107880,128747,0,31,11,341
57,2022-08,Kia K5,Comfort,13,0,0,403,<NA>,0,2900,1740,0,119349,0,31,13,403
58,2022-09,Kia K5,Comfort,13,0,0,390,296,0,2900,1740,0,136298,0,30,13,390
59,2022-10,Kia K5,Comfort,13,0,0,403,334,0,2900,1740,0,132669,0,31,13,403


In [26]:
(1/df.shape[0])*100, 'percent of total data'

(0.8, 'percent of total data')

In [27]:
df.shape

(125, 17)

Так как у нас только один пропуск, который хоть и важный для нашей задачи, затрпгивает только 0.8%, соответсвенно можем удалить без серьезных поторь информации.

In [28]:
df = df.drop(57)
df = df.reset_index(drop=True)

In [29]:
df.shape

(124, 17)

In [30]:
rows_missing = df[df.isnull().any(axis=1)]
rows_missing

,dt,car_model,car_type,car_qnty,new_cars,bad_cars,days available,days on line,days_in_repair,price per day,repair price,money_outcome_repairs,money_outcome_other,new_cars_outcome,dim,active_cars,days_available


## 5. Проверка дубликатов

Данные у нас представляют отчетность моделей машин за кажных месяц, проверка на дублей

In [31]:
df[df[['dt','car_model']].duplicated()]

,dt,car_model,car_type,car_qnty,new_cars,bad_cars,days available,days on line,days_in_repair,price per day,repair price,money_outcome_repairs,money_outcome_other,new_cars_outcome,dim,active_cars,days_available


In [32]:
df.drop(['days_available'],axis=1)

,dt,car_model,car_type,car_qnty,new_cars,bad_cars,days available,days on line,days_in_repair,price per day,repair price,money_outcome_repairs,money_outcome_other,new_cars_outcome,dim,active_cars
0,2022-01,Logan,Econom,31,1,3,868,711,93,2000,1500,139500,128733,400000,31,28
1,2022-02,Logan,Econom,32,1,5,756,740,140,2000,1500,210000,103580,400000,28,27
2,2022-03,Logan,Econom,33,1,1,992,972,31,2000,1500,46500,141270,400000,31,32
3,2022-04,Logan,Econom,34,0,9,750,622,270,2000,1500,405000,101438,0,30,25
4,2022-05,Logan,Econom,34,0,1,1023,849,31,2000,1500,46500,132209,0,31,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,2023-09,E200,Business,13,0,1,360,280,30,10000,31000,930000,141444,0,30,12
120,2023-10,E200,Business,13,0,0,403,290,0,10000,31000,0,148489,0,31,13
121,2023-11,E200,Business,13,0,1,360,277,30,10000,31000,930000,128343,0,30,12
122,2023-12,E200,Business,13,0,1,372,286,31,10000,31000,961000,109353,0,31,12


## 6. Проверка выбросов

Тут мы ищем явные выбросы, которые больше подходят как не правильно введеные значения; 
можно было заметить что в `days avaiable` у нас в данный мелькнуло значение которая явно не соответсвует действительностью, явное значие тут будет 930 (dim * active_cars)

In [33]:
df[df['days available'] == 100000]

,dt,car_model,car_type,car_qnty,new_cars,bad_cars,days available,days on line,days_in_repair,price per day,repair price,money_outcome_repairs,money_outcome_other,new_cars_outcome,dim,active_cars,days_available
10,2022-11,Logan,Econom,34,0,3,100000,818,90,2000,1500,135000,128023,0,30,31,930


In [34]:
df.loc[10,'days available'] = 930

In [35]:
df[df['days available'] == 100000]

,dt,car_model,car_type,car_qnty,new_cars,bad_cars,days available,days on line,days_in_repair,price per day,repair price,money_outcome_repairs,money_outcome_other,new_cars_outcome,dim,active_cars,days_available


## 7. Логические проверки

Мы так же можем сделать паку логических проверок 

- `days on line` точно не может быть больше чем `days available`
- Удостоверимя что у нас нет отрицательных значении где они не должны быть (прибль и доходы)

In [49]:
df[(df['days on line'] > df['days available']) == True]

,dt,car_model,car_type,car_qnty,new_cars,bad_cars,days available,days on line,days_in_repair,price per day,repair price,money_outcome_repairs,money_outcome_other,new_cars_outcome,dim,active_cars,days_available
98,2024-01,JAC J7,Comfort,32,0,3,899,1627,93,2100,1721,160053,193870,0,31,29,899


In [51]:
df.describe()

,car_qnty,new_cars,bad_cars,days available,days on line,days_in_repair,price per day,repair price,money_outcome_repairs,money_outcome_other,new_cars_outcome,dim,active_cars,days_available
count,124.0,124.0,124.0,124.000000,124.0,124.0,124.0,124.0,124.0,124.0,124.0,124.000000,124.0,124.0
mean,21.733871,0.532258,1.725806,609.459677,535.83871,52.620968,3761.298387,6598.790323,111403.322581,141258.120968,789677.419355,30.435484,20.008065,609.459677
std,11.922588,0.999475,1.980961,342.881724,337.977543,60.400371,2913.644202,10390.082834,185512.127075,26136.491371,1796727.800588,0.857695,11.202857,342.881724
min,2.0,0.0,0.0,31.000000,24.0,0.0,100.0,810.0,0.0,100864.0,0.0,28.000000,1.0,31.0
25%,13.0,0.0,0.0,360.000000,265.0,0.0,2000.0,1500.0,0.0,122975.75,0.0,30.000000,12.0,360.0
50%,20.0,0.0,1.0,529.500000,495.5,31.0,2400.0,1740.0,71952.0,135475.5,0.0,31.000000,18.0,529.5
75%,30.0,1.0,2.0,837.000000,778.5,62.0,3200.0,2011.0,114571.5,156763.75,400000.0,31.000000,27.0,837.0
max,48.0,5.0,9.0,1457.000000,1627.0,279.0,10001.0,32000.0,992000.0,197857.0,9000000.0,31.000000,47.0,1457.0


Аналогично с пропуском в подвыборке **Kia K5**, такое количество строк можно удалить судя по рекоммендациям, так как количество не рпывышает 10% для подвыборки этой модели **JAC J7**

In [52]:
df = df.drop(98)
df = df.reset_index(drop=True)

# Экспотр готового датасата

In [53]:
df = df.drop(['days_available'],axis=1)

In [54]:
df.to_csv('car_rental.csv',index=False)